In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split

development = pd.read_csv("./../../DSL_Winter_Project_2024/development.csv")

outlier_column_index=[0, 7, 12, 15, 16, 17]
columns_to_drop=[]

for index in outlier_column_index:
    columns_to_drop.append('pmax[%s]' % index)
    columns_to_drop.append('negpmax[%s]' % index)
    columns_to_drop.append('tmax[%s]' % index)
    columns_to_drop.append('area[%s]' % index)
    columns_to_drop.append('rms[%s]' % index)

evaluation = pd.read_csv("./../../DSL_Winter_Project_2024/evaluation.csv")
eva=evaluation.drop(columns=columns_to_drop)
eva_df=eva.drop(columns=["Id"])

df=development.drop(columns=columns_to_drop)
X=df.drop(columns=['x', 'y'])
y=df.loc[:,['x', 'y']]

rs=42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rs)

In [22]:
import keras
from keras import layers
from keras_tuner import RandomSearch

In [23]:
def build_model(hp):
    model = keras.Sequential()
    
    model.add(layers.Input(shape=(X_train.shape[1],)))
    for i in range(hp.Int('num_layers', min_value=1, max_value=5, step=1)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
                               activation='sigmoid'))
        model.add(layers.Dropout(rate=hp.Float('dropout_' + str(i), min_value=0.0, max_value=0.5, step=0.1)))

    model.add(layers.Dense(units=2))

    model.compile(optimizer=keras.optimizers.legacy.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
                  loss='mse',
                  metrics=['mae', 'acc'])
    return model


In [24]:
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=50,
    executions_per_trial=3,
    overwrite=True,
    directory='my_tuning_directory',
    project_name='my_first_tuning_project'
)

In [25]:
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Trial 22 Complete [00h 13m 39s]
val_loss: 33.81149164835612

Best val_loss So Far: 25.951648076375324
Total elapsed time: 13h 36m 54s

Search: Running Trial #23

Value             |Best Value So Far |Hyperparameter
1                 |4                 |num_layers
384               |480               |units_0
0.4               |0.3               |dropout_0
0.00019167        |0.00018093        |learning_rate
480               |96                |units_1
0.3               |0                 |dropout_1
96                |160               |units_2
0.1               |0.1               |dropout_2
192               |96                |units_3
0.1               |0.1               |dropout_3
64                |256               |units_4
0.3               |0.4               |dropout_4

Epoch 1/50
9638/9638 [==============================] - 6s 578us/step - loss: 61189.4531 - mae: 205.5058 - acc: 0.5173 - val_loss: 12498.0029 - val_mae: 94.9019 - val_acc: 0.5841
Epoch 2/50
9638/9638 [============

KeyboardInterrupt: 

In [10]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [11]:
best_hyperparameters

{'num_layers': 3,
 'units_0': 64,
 'dropout_0': 0.1,
 'learning_rate': 0.00012162727501515752,
 'units_1': 32,
 'dropout_1': 0.0,
 'units_2': 448,
 'dropout_2': 0.0}

In [20]:
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def build_model(activation='sigmoid', optimizer='adam'):
    model = keras.Sequential([
        layers.Input(shape=(X_train.shape[1],)),
        layers.Dense(512, activation=activation),
        layers.Dense(128, activation=activation),
        layers.Dense(64, activation=activation),
        layers.Dense(32, activation=activation),
        layers.Dense(2)
    ])
    model.compile(optimizer=optimizer, loss="mse", metrics=["mae", "acc"])
    return model


model = build_model()
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

y_pred = model.predict(X_test_scaled)

distance=np.mean(np.sqrt(np.sum((y_pred - y_test) ** 2, axis=1)))
print(distance)

Epoch 1/50
7710/7710 [==============================] - 7s 892us/step - loss: 116855.5000 - mae: 317.6105 - acc: 0.5209 - val_loss: 68227.0859 - val_mae: 233.4563 - val_acc: 0.5218
Epoch 2/50
7710/7710 [==============================] - 7s 856us/step - loss: 37385.9648 - mae: 159.8214 - acc: 0.4982 - val_loss: 17377.2930 - val_mae: 110.5212 - val_acc: 0.5218
Epoch 3/50
7710/7710 [==============================] - 7s 858us/step - loss: 12354.1582 - mae: 92.6348 - acc: 0.6011 - val_loss: 6243.1479 - val_mae: 63.1730 - val_acc: 0.9757
Epoch 4/50
7710/7710 [==============================] - 7s 862us/step - loss: 3544.8650 - mae: 42.8766 - acc: 0.9842 - val_loss: 1538.3059 - val_mae: 25.3227 - val_acc: 0.9843
Epoch 5/50
7710/7710 [==============================] - 7s 894us/step - loss: 660.7171 - mae: 14.4470 - acc: 0.9845 - val_loss: 191.6237 - val_mae: 7.5354 - val_acc: 0.9849
Epoch 6/50
7710/7710 [==============================] - 7s 895us/step - loss: 83.6166 - mae: 5.3404 - acc: 0.9855

In [ ]:
eva_df_scaled = scaler.transform(eva_df)

eva_y=model.predict(eva_df_scaled)

df = pd.DataFrame(eva_y, columns=['Id', 'Predicted'])
df['Predicted'] = df.apply(lambda row: f"{row['Id']:.1f}|{row['Predicted']:.1f}", axis=1)
df['Id'] = df.index
df.to_csv('tensor_output4.csv', index=False, sep=',')